In [1]:
import numpy as np
def indicator_fill(df):
    a = ta.trend.ema_indicator(close=df.Close,window=5)
    f = ta.trend.sma_indicator(close=df.Close,window=50)
    g = ta.trend.sma_indicator(close=df.candle_sign,window=20)
    i = ta.trend.sma_indicator(close=df.Close,window=20)
    k = ta.trend.sma_indicator(close=df.Close,window=10)
    ssl_high = ta.trend.sma_indicator(close=df.High,window=100)
    ssl_low = ta.trend.sma_indicator(close=df.Low,window=100)
    l = ta.trend.ichimoku_base_line(high=df.High,low=df.Low)
    m = ta.trend.ichimoku_conversion_line(high=df.High,low=df.Low)
    h = ta.trend.sma_indicator(close=df.ADF,window=20)
    j = ta.trend.sma_indicator(close=df.dur,window=20)
    v = ta.volume.chaikin_money_flow(high=df.High,low=df.Low,close=df.Close,volume=df.Volumer)
    c = ta.volatility.average_true_range(high=df.High,low=df.Low,close=df.Close)
    df["5_EMA"] = a
    df["50_SMA"] = f
    df["20_SMA"] = i
    df["10_SMA"] = k
    df["candle_sign_agg"] = g
    df["ATR"] = c
    df["ADF_MA"] = h
    df["DUR_MA"] = j
    df["kijun"] = l
    df["tenkan"] = m
    df["chaikin"] = v
    df["ssl_high"] = ssl_high
    df["ssl_low"] = ssl_low
    return df


In [2]:
def upper_shadow(opener, close, high):
    if opener >= close:
        return abs(opener-high)
    else:
        return abs(close-high)

def lower_shadow(opener, close, low):
    if opener <= close:
        return abs(opener-low)
    else:
        return abs(close-low)

def real_body(opener, close):
    return abs(opener-close)

def candle_preprocess(df):

    df["rb"] = df.apply(lambda x: real_body(x.Open,x.Close), axis = 1)
    df["us"] = df.apply(lambda x: upper_shadow(x.Open,x.Close,x.High), axis = 1)
    df["ls"] = df.apply(lambda x: lower_shadow(x.Open,x.Close,x.Low), axis = 1)
    return df
    

In [3]:
from fracdiff import FracdiffStat
import pickle
from joblib import dump
from joblib import load
def frac_diff_calc(df):
    from joblib import load
    f = load('frac_diff_SPX.DAT')
    at_f = load('frac_diff_SPX_ATR.DAT')
    df = df[50:]
    c = np.array(df.Close).reshape(-1,1)
    e = np.array(df["5_EMA"]).reshape(-1,1)
    s = np.array(df["50_SMA"]).reshape(-1,1)
    k = np.array(df["20_SMA"]).reshape(-1,1)
    a = np.array(df["ATR"]).reshape(-1,1)
    df["Close"] = list(np.squeeze(f.transform(c), axis = 1))
    df["5_EMA"] = list(np.squeeze(f.transform(e), axis = 1))
    df["50_SMA"] = list(np.squeeze(f.transform(s), axis = 1))
    df["20_SMA"] = list(np.squeeze(f.transform(k), axis = 1))
    df["ATR"] = list(np.squeeze(at_f.transform(a), axis = 1))
    df = df[1:]
    return df

In [8]:
def df_vectorize(df):
    vec_load = []
    side = []
    meta = []
    buff_df = df[len(df)-50:len(df)]
    close_log = buff_df.Close.tolist()
    tick_adf = buff_df.ADF.tolist()
    adf_ma = buff_df.ADF_MA.tolist()
    c_adf = buff_df.Candle_ADF.tolist()
    tick_sign = buff_df.tick_sign_agg.tolist()
    candle_sign = buff_df.candle_sign_agg.tolist()
    c_sign = buff_df.candle_sign.tolist()
    rb_vec = buff_df.rb.tolist()
    us_vec = buff_df.us.tolist()
    ls_vec = buff_df.ls.tolist()
    ema_5 = buff_df["5_EMA"].tolist()
    sma_50 = buff_df["50_SMA"].tolist()
    sma_20 = buff_df["20_SMA"].tolist()
    dur = buff_df.dur.tolist()
    dur_ma = buff_df.DUR_MA.tolist()
    # interaction features ###############################################################
    ma_diff = list(np.array(ema_5)-np.array(sma_50))
    ma_20_diff = list(np.array(ema_5)-np.array(sma_20))
    close_diff = list(np.array(close_log)-np.array(sma_50))
    atr_vec = list(np.array(buff_df.ATR.tolist()) * np.array(close_log))
    tick_close_interact = list(np.array(tick_sign) * np.array(close_log))
    candle_sign_ma_interact = list(np.array(candle_sign) * np.array(sma_50))
    c_sign_close_interact = list(np.array(c_sign) * np.array(close_log))
    # This 50 lookback v4
    #vec_load += [tick_adf + tick_sign + c_sign + rb_vec + us_vec + ls_vec]
    # This 200 lookback v3
    vec_load += [tick_adf + c_adf + tick_sign + candle_sign + rb_vec + us_vec + ls_vec
                + ma_diff + close_diff + atr_vec + adf_ma]
    # This v4
    #vec_load += [close_log + tick_adf + c_adf + tick_sign + candle_sign + rb_vec + us_vec + ls_vec + ema_5 + sma_50
    #            + ma_diff + close_diff + atr_vec + adf_ma + tick_close_interact + candle_sign_ma_interact]
    #vec_load += [close_log + tick_adf + c_adf + tick_sign + candle_sign + rb_vec + us_vec + ls_vec + ema_5 + sma_50 + sma_20
    #            + ma_diff + ma_20_diff + close_diff + atr_vec + adf_ma + tick_close_interact + candle_sign_ma_interact]
    #vec_load += [c_adf + tick_sign + rb_vec + us_vec + ls_vec + ema_5 + sma_50
    #            + ma_diff + close_diff + adf_ma]
    #vec_load += [close_log + tick_sign + rb_vec + us_vec + ls_vec + ema_5 + sma_50
    #            + sma_20]
    return vec_load

In [6]:
import ta
import time
import datetime
import pandas as pd
trade_df = pd.DataFrame({"a":[2]})
trade_df.to_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\AE925E3426D43DD22F06664D804D4525\MQL4\Files\trade_spx.csv",index=False)
close_df = pd.DataFrame({"a":[2]})
close_df.to_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\AE925E3426D43DD22F06664D804D4525\MQL4\Files\closure_spx.csv",index=False)
ind_df = pd.DataFrame({"Ind":[0]})
ind_df.to_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\F354DCCD667FB40D8D6DE5A4406D8652\MQL4\Files\load_ind_spx.csv",index=False)
opener = []
high = []
low = []
close = []
volume = []
tick_adf = []
candle_adf = []
sign_agg = []
pos_run_agg = []
counter = 0
candle_form = 0
dt = datetime.datetime.now()
dur = []

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import ta
import time
import pandas as pd
from joblib import load
from statsmodels.tsa.stattools import adfuller
from IPython.display import clear_output, display

model_1 = load("200_lookback_fracdiff_side_prediction_xgbclasser_interaction_features_SPX_v3.model")

u=2

while True:
    try:
        ind_df = pd.read_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\F354DCCD667FB40D8D6DE5A4406D8652\MQL4\Files\load_ind_spx.csv", sep=";",names=["Ind"])
    except:
        continue
    if ind_df.Ind.tolist()[0] == 1:
        try:
            df = pd.read_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\F354DCCD667FB40D8D6DE5A4406D8652\MQL4\Files\your_data_spx.csv",sep=";", names=["Price"])
        except:
            continue
        ind_df = pd.DataFrame({"Ind":[0]})
        ind_df.to_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\F354DCCD667FB40D8D6DE5A4406D8652\MQL4\Files\load_ind_spx.csv",index=False)
        
        neg_run_agg = []
        price = df.Price.tolist()
        buff = 0
        start = 0
        neg_run = 0
        pos_run = 0
        
        
        
        prev = True
        for i in range(len(price)):
            #buff += (doll[i]*vol[i])
            buff += 1
            if buff>=100:
                clear_output(wait=True)
                opener += [price[start]]
                high += [max(price[start:i+1])]
                low += [min(price[start:i+1])]
                close += [price[i]]
                tick_adf += [adfuller(price[start:i+1])[1]]
                dur += [(datetime.datetime.now()-dt).total_seconds()]
                dt = datetime.datetime.now()
                counter += 1
                print(len(close))
                if counter >= 20:
                    candle_adf += [adfuller(close[counter-20:counter])[1]]
                else:
                    candle_adf += ["pass"]
                if len(close) > 1:
                    if close[-1] < close[-2]:
                        sign_agg += [-1]
                        prev = False
                    elif price[-1] > price[-2]:
                        sign_agg += [1]
                        prev = True
                    elif prev:
                        sign_agg += [1]
                    else:
                        sign_agg += [-1]
                else:
                    sign_agg += [1]
                for k in range(start+1,i+1):
                    if price[k] < price[k-1]:
                        pos_run -= 1
                        prev = False
                    elif price[k] > price[k-1]:
                        pos_run += 1
                        prev = True
                    elif prev:
                        pos_run += 1
                    else:
                        pos_run -= 1
                pos_run_agg += [pos_run/100.0]
                start = i+1
                buff = 0



    if len(close) > 110:
        volumer = [200] * len(close)
        candle_form = 1
        candle_data = pd.DataFrame({"Volumer":volumer,"Open":opener,"High":high,"Low":low,"Close":close,"ADF":tick_adf,"Candle_ADF":candle_adf,"tick_sign_agg":pos_run_agg,"candle_sign":sign_agg,"dur":dur})
        candle_data = candle_preprocess(candle_data)
        candle_data = indicator_fill(candle_data)
        atr = candle_data.ATR.tolist()
        fema = candle_data["tenkan"].tolist()
        #fema = candle_data["5_EMA"].tolist()
        tosima = candle_data["20_SMA"].tolist()
        kij = candle_data["kijun"].tolist()
        chaikin = candle_data["chaikin"].tolist()
        ssl_hi = candle_data["ssl_high"].tolist()
        ssl_lo = candle_data["ssl_low"].tolist()
        #candle_data = frac_diff_calc(candle_data)
        v = df_vectorize(candle_data)
        test_v = np.array(v)
        #preds = model_1.predict(test_v)
        preds = [3]
        #probs = np.max(model_1.predict_proba(test_v),axis=1)
        probs = [3]
        if (close[-2] <= ssl_hi[-2] or close[-2] <= ssl_lo[-2]) and (close[-1] > ssl_hi[-1] and close[-1] > ssl_lo[-1]):
            u = 0
        elif (close[-2] >= ssl_hi[-2] or close[-2] >= ssl_lo[-2]) and (close[-1] < ssl_hi[-1] and close[-1] < ssl_lo[-1]):
            u = 1
        else:
            u = 3
        if u == 0:
            p = 0
        elif u == 1:
            p = 1
        else:
            p = 2
        try:
            trade_df = pd.DataFrame({"a":preds})
            prob_df = pd.DataFrame({"a":probs})
            ind_cross = pd.DataFrame({"a":[p]})
            ind_cross.to_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\AE925E3426D43DD22F06664D804D4525\MQL4\Files\ind_cross_spx.csv",index=False)
            u_cross = pd.DataFrame({"a":[u]})
            u_cross.to_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\AE925E3426D43DD22F06664D804D4525\MQL4\Files\u_cross_spx.csv",index=False)
            trade_df.to_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\AE925E3426D43DD22F06664D804D4525\MQL4\Files\trade_spx.csv",index=False)
            prob_df.to_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\AE925E3426D43DD22F06664D804D4525\MQL4\Files\prob_spx.csv",index=False)
            atr_df = pd.DataFrame({"a":[atr[-1]]})
            atr_df.to_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\AE925E3426D43DD22F06664D804D4525\MQL4\Files\atr_spx.csv",index=False)
            close_df = pd.DataFrame({"a":[candle_form]})
            close_df.to_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\AE925E3426D43DD22F06664D804D4525\MQL4\Files\closure_spx.csv",index=False)
        except:
            continue
        close.pop(0)
        low.pop(0)
        high.pop(0)
        opener.pop(0)
        tick_adf.pop(0)
        candle_adf.pop(0)
        pos_run_agg.pop(0)
        sign_agg.pop(0)
        dur.pop(0)
        counter -= 1

    time.sleep(1)


    
    
    
    
    
    
    

111


In [7]:
from time import sleep
from IPython.display import clear_output, display

for f in range(10):
    clear_output(wait=True)
    print(f)  # use display(f) if you encounter performance issues
    sleep(1)

9


In [ ]:
        if fema[-2] < tosima[-2] and fema[-1] > tosima[-1]:
            p = 0
        elif fema[-2] > tosima[-2] and fema[-1] < tosima[-1]:
            p = 1
        else:
            p = 2
        
        if fema[-2] <= kij[-2] and fema[-1] > kij[-1]:
            u = 0
        elif fema[-2] >= kij[-2] and fema[-1] < kij[-1]:
            u = 1
        if u == 0 and close[-1] > high[-27]:
            p = 0
        elif u == 1 and close[-1] < low[-27]:
            p = 1
        else:
            p = 2

In [4]:
import pandas as pd

dfer = pd.read_csv("50_GBP_v4checkv16.csv")

can_buy = True
can_sell = True
sold = False
bought = False
sale = []
bcount = 0
scount = 0
for i,j in dfer.iterrows():
    if j.Pred == "buy" and can_buy:
        sale += ["bought"]
        can_buy = False
        bought = True
        bcount = 0
    elif j.Pred == "sell" and can_sell:
        sale += ["sold"]
        can_sell = False
        sold = True
        scount = 0
    else:
        sale += ["Ongoing"]
    if bcount >= 20:
        can_buy = True
        bought = False
    if bought:
        bcount += 1
    if scount >= 20:
        can_sell = True
        sold = False
    if sold:
        scount += 1

dfer["Sales"] = sale
dfer.to_csv("50_GBP_v4checkv16.csv", index = False)